# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,palikir - national government center,6.9248,158.1611,84.94,79,100,11.50,FM,1715210652
1,1,edinburgh of the seven seas,-37.0676,-12.3116,58.17,66,84,26.06,SH,1715210652
2,2,sinabang,2.4803,96.3801,80.56,85,79,4.38,ID,1715210652
3,3,grytviken,-54.2811,-36.5092,27.37,76,9,8.61,GS,1715210652
4,4,thompson,55.7435,-97.8558,59.16,33,75,5.75,CA,1715210652


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
configued_data = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["Lat", "Lng"]
)
configued_data
# Display the map
configued_data

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
filtered_cities = city_data_df.loc[(city_data_df['Max Temp'] > 10) & (city_data_df['Humidity'] < 50)]

# Drop any rows with null values
filtered_cities = city_data_df.loc[(city_data_df['Max Temp'] > 10) & (city_data_df['Humidity'] < 50)]
clean_city_data = filtered_cities.dropna()

# Display sample data
# YOUR CODE HERE
clean_city_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,thompson,55.7435,-97.8558,59.16,33,75,5.75,CA,1715210652
6,6,malkapur,20.8833,76.2000,86.36,46,3,10.87,IN,1715210652
18,18,smithers,54.7804,-127.1743,59.32,31,20,8.05,CA,1715210654
22,22,adis zemen,12.1167,37.7833,70.07,45,89,2.59,ET,1715210654
24,24,embu guacu,-23.8322,-46.8114,80.91,44,6,4.00,BR,1715210412
...,...,...,...,...,...,...,...,...,...,...
539,539,port hedland,-20.3167,118.5667,69.85,35,0,9.22,AU,1715210720
548,548,balykchy,42.4557,76.1861,52.02,48,60,0.96,KG,1715210721
558,558,wau,7.7029,27.9953,84.58,36,58,4.43,SS,1715210722
566,566,jinzhou,41.1078,121.1417,68.47,24,100,11.56,CN,1715210723


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = clean_city_data[['City', 'Country', 'Lat', 'Lng', 'Humidity']]
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""
# Display sample data
# YOUR CODE HERE
hotel_df.head()
# Assuming you have a DataFrame called city_data_df with columns 'City', 'Country', 'Latitude', 'Longitude', and 'Humidity'


/var/folders/gv/0nrt3lxs1vl5d5bcbmv5lxqh0000gn/T/ipykernel_1217/338584814.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=""


,City,Country,Lat,Lng,Humidity,Hotel Name
4,thompson,CA,55.7435,-97.8558,33,
6,malkapur,IN,20.8833,76.2000,46,
18,smithers,CA,54.7804,-127.1743,31,
22,adis zemen,ET,12.1167,37.7833,45,
24,embu guacu,BR,-23.8322,-46.8114,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":  "3e0e1a2861fc4b8ea507816659537f4c",  # Add your API key here
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data hotel_df using Jupyter Notebook


Starting hotel search
thompson - nearest hotel: Thompson Inn
malkapur - nearest hotel: No hotel found
smithers - nearest hotel: Sunshine Inn Hotel
adis zemen - nearest hotel: No hotel found
embu guacu - nearest hotel: Centro Paulus
badger - nearest hotel: Clarion Hotel & Suites
whitehorse - nearest hotel: Town & Mountain Hotel
bardai - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
aleksandrovsk-sakhalinskiy - nearest hotel: Три брата
kyabe - nearest hotel: No hotel found
tabas - nearest hotel: هتل امیر
campina verde - nearest hotel: No hotel found
winnemucca - nearest hotel: The Overland Hotel
la ronge - nearest hotel: La Ronge Hotel and Suites
chisec - nearest hotel: bombil pek
victorica - nearest hotel: Hotel Mazamet
fort st. john - nearest hotel: Fort St. John Motor Inn
portland - nearest hotel: the Hoxton
tlatlauquitepec - nearest hotel: Hotel Serrano
kassala - nearest hotel: No hotel found
jishu - nearest hotel: No hotel found
al 'amarah - near

In [12]:
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,thompson,CA,55.7435,-97.8558,33,Thompson Inn
6,malkapur,IN,20.8833,76.2000,46,No hotel found
18,smithers,CA,54.7804,-127.1743,31,Sunshine Inn Hotel
22,adis zemen,ET,12.1167,37.7833,45,No hotel found
24,embu guacu,BR,-23.8322,-46.8114,44,Centro Paulus
...,...,...,...,...,...,...
539,port hedland,AU,-20.3167,118.5667,35,The Esplanade Hotel
548,balykchy,KG,42.4557,76.1861,48,Lake hotel
558,wau,SS,7.7029,27.9953,36,Safari Park Hotel
566,jinzhou,CN,41.1078,121.1417,24,金厦国际饭店


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
hotel_map_df = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["Lat", "Lng"]
)
# Display the map
hotel_map_df

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)